In [17]:
import requests
import csv
import pandas as pd
import datetime
import time
import os, errno
import os.path
from sqlalchemy import exc
from sqlalchemy import create_engine
import warnings


'''
SETTING VARIABLES
'''

#1. DATABASE
user_name = 'root'
address = 'localhost'
port = '3306'
db_name = 'TWSE'



#2. TWSE DATA

#The earliest data datestamp to download for first time use
default_start_date = datetime.datetime(2012,1,1)


#Global Variables
engine=create_engine("mysql+pymysql://" + user_name + ":@" + address + ":" + port + "/" + db_name + "?charset=utf8",echo=False)
engine_int=create_engine("mysql+pymysql://" + user_name + ":@" + address + ":" + port + "/" + "?charset=utf8",echo=False)

In [18]:
'''
initialize mysql environment for first time use
1. Create Database if not exists
2. Create Tables if not exists
'''

def init_mysql_env():

    db = 'CREATE DATABASE IF NOT EXISTS `' + db_name + '` /*!40100 DEFAULT CHARACTER SET big5 */;'

    tbl1 = '''
        CREATE TABLE IF NOT EXISTS '''+ db_name + '''.`個股日成交資訊` (
      `日期` datetime NOT NULL,
      `證券代號` varchar(50) NOT NULL,
      `證券名稱` varchar(100) NOT NULL,
      `成交股數` bigint(20) DEFAULT NULL,
      `成交筆數` bigint(20) DEFAULT NULL,
      `成交金額` bigint(20) DEFAULT NULL,
      `開盤價` double DEFAULT NULL,
      `最高價` double DEFAULT NULL,
      `最低價` double DEFAULT NULL,
      `收盤價` double DEFAULT NULL,
      `漲跌價差` double DEFAULT NULL,
      `最後揭示買價` double DEFAULT NULL,
      `最後揭示買量` double DEFAULT NULL,
      `最後揭示賣價` double DEFAULT NULL,
      `最後揭示賣量` double DEFAULT NULL,
      `本益比` double DEFAULT NULL,
      PRIMARY KEY (`證券代號`,`日期`,`證券名稱`),
      KEY `ix_個股日成交資訊_日期` (`日期`)
    ) ENGINE=InnoDB DEFAULT CHARSET=big5;
    '''

    tbl2 = '''
    CREATE TABLE IF NOT EXISTS '''+ db_name + '''.`大盤統計資料_報酬指數` (
      `日期` datetime NOT NULL,
      `報酬指數` varchar(100) NOT NULL,
      `收盤指數` double DEFAULT NULL,
      `漲跌點數` double DEFAULT NULL,
      `漲跌百分比` double DEFAULT NULL,
      PRIMARY KEY (`報酬指數`,`日期`),
      KEY `ix_大盤統計資料_報酬指數_日期` (`日期`)
    ) ENGINE=InnoDB DEFAULT CHARSET=big5;
    '''

    tbl3 = '''
    CREATE TABLE IF NOT EXISTS '''+ db_name + '''.`大盤統計資料_指數` (
      `日期` datetime NOT NULL,
      `指數` varchar(100) NOT NULL,
      `收盤指數` double DEFAULT NULL,
      `漲跌點數` double DEFAULT NULL,
      `漲跌百分比` double DEFAULT NULL,
      PRIMARY KEY (`指數`,`日期`),
      KEY `ix_大盤統計資料_指數_日期` (`日期`)
    ) ENGINE=InnoDB DEFAULT CHARSET=big5;
    '''

    tbl4 = '''
    CREATE TABLE IF NOT EXISTS '''+ db_name + '''.`成交統計` (
      `日期` datetime NOT NULL,
      `成交統計` varchar(100) NOT NULL,
      `成交金額(元)` bigint(20) DEFAULT NULL,
      `成交股數(股)` bigint(20) DEFAULT NULL,
      `成交筆數` bigint(20) DEFAULT NULL,
      PRIMARY KEY (`成交統計`,`日期`),
      KEY `ix_成交統計_日期` (`日期`)
    ) ENGINE=InnoDB DEFAULT CHARSET=big5;
    '''

    #To ignore ALREADY EXISTS warnings
    warnings.simplefilter('ignore')
    conn = engine_int.connect()
    conn.execute(db)
    conn.execute(tbl1)
    conn.execute(tbl2)
    conn.execute(tbl3)
    conn.execute(tbl4)
    conn.close()
    warnings.simplefilter('default')

In [19]:

#to remove speical char in downloaded contents
def df_clean(df):
    df = df.replace('--',0)
    df = df.replace('---',0)
    df = df.replace(',','', regex=True)
    df = df.replace('\"','', regex=True)
    df = df.replace('=','', regex=True)
    df = df.replace(' ','', regex=True) 
    return df


def left(s, amount):
    return s[:amount]

def right(s, amount):
    return s[-amount:]

def mid(s, offset, amount):
    return s[offset:offset+amount]


#Fetch the last existing data datastamp
def get_last_day(DB_Path):
    sql = "SELECT max(日期) as last_day FROM " + DB_Path + ";"
    try:
        df_tmp = pd.read_sql(sql,engine)
        day = df_tmp['last_day'][0]
        print('The last date is', day)
    except:
        day = None
    return day


#download and store the data on MySQL tables
def get_daily_market_closing(file_type):
    last_date = get_last_day('大盤統計資料_指數')
    end_date = datetime.datetime.today() - datetime.timedelta(days=1)
    
    if last_date is None:
        start_date = default_start_date
    else:
        start_date = last_date + datetime.timedelta(days=1)
    
    while start_date < end_date:
        year = str(start_date.year)
        month = right('00' + str(start_date.month),2)
        day = right('00' + str(start_date.day),2)
    
        try:
            dls = "http://www.tse.com.tw/exchangeReport/MI_INDEX?response=csv&date=" + year + month + day + "&type=" + file_type
            resp = requests.get(dls)
            resp.encoding ='big5'
            output = open('tmp.csv', 'w')
            output.write(resp.text)
            output.close()
            
            #ALLBUT0999 = 全部(不含權證, 牛熊證, 可展延牛熊證)
            if file_type == 'ALLBUT0999':
                store_daily_market_closing(start_date)
            print("imported 各類指數日成交量值 " + str(start_date))
           
            os.remove("tmp.csv")
        except IndexError:
            print("no data on " + str(start_date))
            start_date = start_date + datetime.timedelta(days=1)
            continue        
        
        start_date = start_date + datetime.timedelta(days=1)


def store_daily_market_closing(date):

    
    tbl1 = []
    tbl2 = []
    tbl3 = []
    tbl4 = []
    tbl5 = []
    i = 0
    with open('tmp.csv', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) > 0:
                record = [date]
                
                if row[0] == '指數':
                    i += 1
                elif row[0] == '報酬指數':
                    i += 1
                elif row[0] == '成交統計':
                    i += 1
                elif row[0] == '漲跌證券數合計':
                    i += 1
                elif row[0] == '類型': 
                    i += 1
                elif row[0] == '備註:':
                    i += 1
                elif row[0] == '證券代號':
                    i += 1
                    
                record.extend(row)
                
                if i == 1:
                    tbl1.append(record)
                elif i == 2:
                    tbl2.append(record)
                elif i == 3:
                    tbl3.append(record)
                #elif i == 5:
                    #tbl4.append(row)  #跳過 "漲跌證券數合計"
                elif i == 7:
                    tbl5.append(record)                            
    
    tbl1_col = ['日期']
    tbl1_col.extend(tbl1[0][1:])
    tbl2_col = ['日期']
    tbl2_col.extend(tbl2[0][1:])
    tbl3_col = ['日期']
    tbl3_col.extend(tbl3[0][1:])
    tbl5_col = ['日期']
    tbl5_col.extend(tbl5[0][1:])

    del tbl1[0]
    del tbl2[0]
    del tbl3[0]
    #del tbl4[0]
    del tbl5[0]

    tbl1_df = pd.DataFrame(tbl1, columns= tbl1_col)
    tbl2_df = pd.DataFrame(tbl2, columns= tbl2_col)
    tbl3_df = pd.DataFrame(tbl3, columns= tbl3_col)
    #tbl4_df = pd.DataFrame(tbl4, columns= tbl4_col)
    tbl5_df = pd.DataFrame(tbl5, columns= tbl5_col)
    
    tbl1_df.drop([''],axis=1, inplace = True)
    tbl2_df.drop([''],axis=1, inplace = True)
    tbl3_df.drop([''],axis=1, inplace = True)
    #tbl4_df.drop([''],axis=1, inplace = True)
    tbl5_df.drop([''],axis=1, inplace = True)

    tbl1_df = df_clean(tbl1_df);
    tbl2_df = df_clean(tbl2_df);
    tbl3_df = df_clean(tbl3_df);
    #tbl4_df = df_clean(tbl4_df);
    tbl5_df = df_clean(tbl5_df);
    
    tbl1_df[['收盤指數','漲跌點數', '漲跌百分比(%)']] = tbl1_df[['收盤指數','漲跌點數', '漲跌百分比(%)']].apply(pd.to_numeric)
    tbl2_df[['收盤指數','漲跌點數', '漲跌百分比(%)']] = tbl2_df[['收盤指數','漲跌點數', '漲跌百分比(%)']].apply(pd.to_numeric)
    tbl3_df[['成交金額(元)','成交股數(股)','成交筆數']] = tbl3_df[['成交金額(元)','成交股數(股)','成交筆數']].apply(pd.to_numeric)
    tbl5_df[['成交股數','成交筆數','成交金額','開盤價','最高價','最低價','收盤價','漲跌價差','最後揭示買價','最後揭示買量','最後揭示賣價','最後揭示賣量','本益比']] = tbl5_df[['成交股數','成交筆數','成交金額','開盤價','最高價','最低價','收盤價','漲跌價差','最後揭示買價','最後揭示買量','最後揭示賣價','最後揭示賣量','本益比']].apply(pd.to_numeric)
      
    tbl1_df = tbl1_df.set_index('日期')
    tbl2_df = tbl2_df.set_index('日期')
    tbl3_df = tbl3_df.set_index('日期')
    tbl5_df = tbl5_df.set_index('日期')
    
    tbl1_df.drop(['漲跌(+/-)'], axis =1, inplace = True)
    tbl2_df.drop(['漲跌(+/-)'], axis =1, inplace = True)
    tbl5_df.drop(['漲跌(+/-)'], axis =1, inplace = True)
    
    tbl1_df.columns = [w.replace('(%)', '') for w in tbl1_df.columns]
    tbl2_df.columns = [w.replace('(%)', '') for w in tbl2_df.columns]
            
    tbl1_df.to_sql(name= '大盤統計資料_指數', con=engine, if_exists = 'append', index=True)
    tbl2_df.to_sql(name= '大盤統計資料_報酬指數', con=engine, if_exists = 'append', index=True)
    tbl3_df.to_sql(name= '成交統計', con=engine, if_exists = 'append', index=True)
    tbl5_df.to_sql(name= '個股日成交資訊', con=engine, if_exists = 'append', index=True)
        
    

In [20]:
init_mysql_env()
get_daily_market_closing("ALLBUT0999")


The last date is 2017-07-31 00:00:00
